## Assess ensemble model performance on validation data for death and to output death predictions

In [7]:
import pickle
import pandas as pd
from sklearn.metrics import recall_score, precision_score, f1_score, roc_auc_score

#import logR death model
logR_death = pickle.load(open('ensb_logreg_death.pkl', 'rb'))

In [5]:
#load validation death probs from text and TCN
validation_death_text = pd.read_csv("text_mortality_probability_validation.csv")
validation_death_text = validation_death_text[['Death', 'icu_death']]
validation_death_TCN = pd.read_csv('tcn_probs_of_death_valid.csv')
validation_death_TCN = validation_death_TCN.rename(columns={"probs_of_death": "TCN_val_death"})
validation_death_combined = pd.concat([validation_death_text, validation_death_TCN],join='outer', axis=1)
validation_death_combined

,Death,icu_death,TCN_val_death
0,0.295472,0,8.307330e-02
1,0.219437,0,1.749867e-06
2,0.709828,0,8.418267e-01
3,0.429824,0,1.616446e-05
4,0.456270,0,1.247955e-02
...,...,...,...
4076,0.433242,0,8.179577e-04
4077,0.784476,1,3.232544e-01
4078,0.649372,0,1.168605e-04
4079,0.077565,0,2.011393e-07


In [6]:
#predict death from text and TCN for validated data, and output
death_predictors = validation_death_combined[['Death','TCN_val_death']]
predicted_death = logR_death.predict(death_predictors)
predicted_death_df = pd.DataFrame(predicted_death)
#predicted_death_df.to_csv('170424_Validation_death.csv')

/Users/wenruiwah/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- Death
- TCN_val_death
Feature names seen at fit time, yet now missing:
- Death_Text
- TCN_death

  warnings.warn(message, FutureWarning)


In [8]:
#scores
print('AUROC:', roc_auc_score(validation_death_combined['icu_death'], predicted_death_df))
print('Precision', precision_score(validation_death_combined['icu_death'], predicted_death_df, average='macro', zero_division=0))
print('Recall', recall_score(validation_death_combined['icu_death'], predicted_death_df, average='macro', zero_division=0))
print('F1 score:', f1_score(validation_death_combined['icu_death'], predicted_death_df, average='macro', zero_division=0))

AUROC: 0.5815613817781448
Precision 0.6640387124836886
Recall 0.5815613817781448
F1 score: 0.6018968582081757
